In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import os
import zipfile
import glob

# Definição do caminho do arquivo ZIP contendo os balanços e do diretório onde os arquivos extraídos serão armazenados
zip_file = '/content/balancos.zip'
extract_folder = '/content/extracted_balancos'

# Criação do diretório de extração, caso ele não exista
os.makedirs(extract_folder, exist_ok=True)

# Extração do conteúdo do arquivo ZIP para a pasta especificada
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Lista com os códigos das empresas cujos balanços e DREs devem ser processados
empresas = ["ABEV3", "AZUL4", "BTOW3", "B3SA3", "BBSE3", "BRML3", "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRFS3", "BPAC11", "CRFB3", "CCRO3", "CMIG4", "HGTX3", "CIEL3", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CVCB3", "CYRE3", "ECOR3", "ELET6", "EMBR3", "ENBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3", "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "NTCO3", "HAPV3", "HYPE3", "IGTA3", "GNDI3", "ITSA4", "ITUB4", "JBSS3", "JHSF3", "KLBN11", "RENT3", "LCAM3", "LAME4", "LREN3", "MGLU3", "MRFG3", "BEEF3", "MRVE3", "MULT3", "PCAR3", "PETR4", "BRDT3", "PRIO3", "QUAL3", "RADL3", "RAIL3", "SBSP3", "SANB11", "CSNA3", "SULA11", "SUZB3", "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VVAR3", "WEGE3", "YDUQ3"]

# Dicionário onde serão armazenados os dados de fundamentos: balanço e DRE de cada empresa
fundamentos = {}

# Lista todos os arquivos extraídos na pasta de balanços
pasta_balancos = os.path.join(extract_folder, 'balancos')
arquivos = os.listdir(pasta_balancos)


# Para cada arquivo, processa os dados se pertencer a uma das empresas desejadas
for arquivo in arquivos:
    # Extrai o código da empresa do nome do arquivo
    nome = arquivo[-9:-4]
    if '11' in nome:
        nome = arquivo[-10:-4]
    if nome in empresas:
        print(nome)
        # pegar o balanço das empresas
        balanco = pd.read_excel(os.path.join(pasta_balancos, arquivo), sheet_name=0)
        # na primeira coluna colocar o titulo com o nome da empresa
        balanco.iloc[0, 0] = nome
        # pegar 1º linha e tornar um cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]
        # tornar a primeira coluna o indice da tabela
        balanco = balanco.set_index(nome)
        # repetindo para o dre
        dre = pd.read_excel(os.path.join(pasta_balancos, arquivo), sheet_name=1)
        dre.iloc[0, 0] = nome
        dre.columns = dre.iloc[0]
        dre = dre[1:]
        dre = dre.set_index(nome)
        fundamentos[nome] = balanco, dre

YDUQ3
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
JHSF3
WARNING *** file size (64288) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64288) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
QUAL3
WARNING *** file size (61185) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (61185) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
FLRY3
WARNING *** file size (64282) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64282) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 

In [ ]:
# Lê o arquivo Excel com as cotações das empresas
cotacoes_df = pd.read_excel('Cotacoes.xlsx')
# Inicializa um dicionário para armazenar os DataFrames de cotações por empresa
cotacoes = {}

# Para cada empresa única no DataFrame de cotações, cria um subconjunto no dicionário
for empresa in cotacoes_df["Empresa"].unique():
    cotacoes[empresa] = cotacoes_df.loc[cotacoes_df['Empresa'] == empresa, :]

# Validação dos dados: remove empresas que possuem valores nulos nas cotações
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        # Remove a empresa tanto do dicionário de cotações quanto do dicionário de fundamentos
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)

# Atualiza a lista de empresas para conter apenas aquelas com dados completos e válidos
empresas = list(cotacoes.keys())

# Exibe a quantidade de empresas que restaram após a filtragem
print(len(empresas))

65


In [ ]:
# Para cada empresa, junta os dados de balanço patrimonial com as respectivas cotações
for empresa in fundamentos:
    tabela = fundamentos[empresa][0].T
    tabela.index = pd.to_datetime(tabela.index, dayfirst=True, errors='coerce')
    tabela_cotacao = cotacoes[empresa].set_index('Date')
    tabela_cotacao = tabela_cotacao[['Adj Close']]
    tabela = tabela.merge(tabela_cotacao, left_index=True, right_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela

# Exibe a tabela consolidada de uma empresa como exemplo
display(fundamentos['ABEV3'])

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,Outros Ativos Circulantes,...,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital,Adj Close
ABEV3,,,,,,,,,,,,,,,,,,,,,
2020-12-31,125196574.72,35342614.528,17090335.744,1700028.032,4303137.792,7605904.896,0,3287141.888,293552,1062513.984,...,57899073.536,54985510.912,0,25920059.392,0,-64989016.064,0,0,0,15.575767
2020-09-30,127056781.312,39098793.984,21660450.816,1442923.008,4156922.88,7341836.8,0,3472880.896,298303.008,725478.976,...,57899073.536,54941057.024,0,20874268.672,5605920.768,-61314039.808,0,0,0,12.159024
2020-06-30,121393004.544,36268126.208,17438951.424,44823,4201583.104,7771618.816,0,5390340.096,356411.008,1064401.984,...,57899073.536,54877110.272,0,20874268.672,2924649.984,-62652346.368,0,0,0,13.710416
2020-03-31,113687560.192,31086356.48,13204471.808,44823,3394298.112,7752655.872,0,5266407.936,447473.984,976225.024,...,57899073.536,54827528.192,0,20874268.672,1391979.008,-64632938.496,0,0,0,11.557861
2019-12-31,101742944.256,27621136.384,11900665.856,14558,4495524.864,5978556.928,0,4074127.104,512532,645171.968,...,57866760.192,54811463.68,0,20874268.672,0,-72274460.672,0,0,0,18.102791
2019-09-30,102475317.248,29004025.856,15016913.92,14304,4236257.024,5978811.904,0,2545863.936,293060,918814.976,...,57866760.192,54779297.792,0,15341367.296,8665335.808,-70900998.144,0,0,0,18.187490
2019-06-30,98738978.816,27356035.072,14233403.392,14038,4103118.08,6197564.928,0,1542556.032,507216,758140.992,...,57800994.816,54811668.48,0,15341367.296,5972242.944,-72301084.672,0,0,0,16.902554
2019-03-31,96905535.488,26194137.088,12822524.928,13772,3980359.936,6032743.936,0,1980342.016,591273.984,773121.024,...,57798844.416,54740389.888,0,15341367.296,3023304.96,-71795384.32,0,0,0,15.901062
2018-12-31,94126137.344,25329604.608,11463497.728,13391,4879256.064,5401793.024,0,2148713.984,741222.016,681731.008,...,57710198.784,54781190.144,0,15434092.544,0,-71584866.304,0,0,0,14.531096


In [ ]:
# Garante que todas as empresas tenham as mesmas colunas que 'ABEV3'
colunas = list(fundamentos['ABEV3'].columns)

for empresa in empresas:
    if set(colunas) != set(fundamentos[empresa].columns):
        fundamentos.pop(empresa)


# Exibe a quantidade final de empresas com dados consistentes
print(len(fundamentos))

61


In [ ]:
# Ajusta colunas duplicadas adicionando sufixos para evitar conflitos
colunas_originais = fundamentos[list(fundamentos.keys())[0]].columns.tolist()
texto_colunas = ';'.join(colunas_originais)

colunas_modificadas = []
for coluna in colunas_originais:
    if colunas_originais.count(coluna) == 2 and coluna not in colunas_modificadas:
        texto_colunas = texto_colunas.replace(';' + coluna + ';', ';' + coluna + '_1;', 1)
        colunas_modificadas.append(coluna)

colunas = texto_colunas.split(';')

# Aplica as colunas ajustadas a todas as empresas, respeitando o número de colunas de cada uma
for empresa in fundamentos:
    n_colunas = len(fundamentos[empresa].columns)
    fundamentos[empresa].columns = colunas[:n_colunas]

In [ ]:
# Conta a quantidade de valores nulos por coluna em todas as empresas
valores_vazios = dict.fromkeys(colunas, 0)
total_linhas = 0
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]
    for coluna in colunas:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()
        valores_vazios[coluna] += qtde_vazios

# Exibe o total de valores nulos por coluna e o total de linhas processadas
print(valores_vazios)
print(total_linhas)

{'Ativo Total': np.int64(0), 'Ativo Circulante': np.int64(0), 'Caixa e Equivalentes de Caixa': np.int64(0), 'Aplicações Financeiras': np.int64(0), 'Contas a Receber_1': np.int64(0), 'Estoques_1': np.int64(0), 'Ativos Biológicos_1': np.int64(0), 'Tributos a Recuperar': np.int64(0), 'Despesas Antecipadas_1': np.int64(0), 'Outros Ativos Circulantes': np.int64(0), 'Ativo Realizável a Longo Prazo': np.int64(0), 'Aplicações Financeiras Avaliadas a Valor Justo': np.int64(0), 'Aplicações Financeiras Avaliadas ao Custo Amortizado': np.int64(0), 'Contas a Receber': np.int64(0), 'Estoques': np.int64(0), 'Ativos Biológicos': np.int64(0), 'Tributos Diferidos_1': np.int64(0), 'Despesas Antecipadas': np.int64(0), 'Créditos com Partes Relacionadas': np.int64(0), 'Outros Ativos Não Circulantes': np.int64(0), 'Investimentos': np.int64(0), 'Imobilizado': np.int64(0), 'Intangível': np.int64(0), 'Diferido': np.int64(0), 'Passivo Total': np.int64(0), 'Passivo Circulante': np.int64(0), 'Obrigações Sociais e 

In [ ]:
# Identifica colunas com muitos valores ausentes para remoção
remover_colunas = []
for coluna in valores_vazios:
    if valores_vazios[coluna] > 50:
        remover_colunas.append(coluna)

# Remove as colunas identificadas e preenche os valores nulos restantes com forward fill
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)
    fundamentos[empresa] = fundamentos[empresa].ffill()

<ipython-input-8-0a38512772ae>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundamentos[empresa] = fundamentos[empresa].ffill()
<ipython-input-8-0a38512772ae>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fundamentos[empresa] = fundamentos[empresa].ffill()
<ipython-input-8-0a38512772ae>:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', Tr

In [ ]:
# Importa NumPy para manipulação de valores nulos
import numpy as np

# Define período de análise
data_inicial = '2012-01-01'
data_final = '2021-12-31'

# Importa dados históricos do Ibovespa (^BVSP) via yfinance
import yfinance as yf
df_ibov = yf.download('^BVSP', start=data_inicial, end=data_final)

# Garante que todas as datas presentes nos fundamentos estejam também no índice df_ibov
datas = fundamentos['ABEV3'].index
for data in datas:
    if data not in df_ibov.index:
        df_ibov.loc[data] = np.nan

# Ordena o índice e preenche valores faltantes (forward fill)
df_ibov = df_ibov.sort_index()
df_ibov = df_ibov.ffill()

# Renomeia coluna 'Close' para 'IBOV'
df_ibov = df_ibov.rename(columns={"Close": "IBOV"})

# Ajusta nome das colunas para o primeiro nível, se MultiIndex
df_ibov.columns = [col[0] for col in df_ibov.columns]

# Faz merge do Ibovespa com os dados de fundamentos de cada empresa
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].merge(
        df_ibov[['IBOV']],
        left_index=True,
        right_index=True,
        how='left'  # Mantém todas as datas dos fundamentos
    )

# Exibe o DataFrame da ABEV3 com o IBOV agregado
display(fundamentos['ABEV3'])

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber_1,Estoques_1,Ativos Biológicos_1,Tributos a Recuperar,Despesas Antecipadas_1,Outros Ativos Circulantes,...,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital,Adj Close,IBOV
ABEV3,,,,,,,,,,,,,,,,,,,,,
2020-12-31,1.251966e+08,3.534261e+07,1.709034e+07,1700028.032,4303137.792,7605904.896,0,3287141.888,293552.000,1062513.984,...,5.498551e+07,0,2.592006e+07,0.000,-6.498902e+07,0.000,0.000,0,15.575767,119306.0
2020-09-30,1.270568e+08,3.909879e+07,2.166045e+07,1442923.008,4156922.880,7341836.800,0,3472880.896,298303.008,725478.976,...,5.494106e+07,0,2.087427e+07,5605920.768,-6.131404e+07,0.000,0.000,0,12.159024,94603.0
2020-06-30,1.213930e+08,3.626813e+07,1.743895e+07,44823.000,4201583.104,7771618.816,0,5390340.096,356411.008,1064401.984,...,5.487711e+07,0,2.087427e+07,2924649.984,-6.265235e+07,0.000,0.000,0,13.710416,95056.0
2020-03-31,1.136876e+08,3.108636e+07,1.320447e+07,44823.000,3394298.112,7752655.872,0,5266407.936,447473.984,976225.024,...,5.482753e+07,0,2.087427e+07,1391979.008,-6.463294e+07,0.000,0.000,0,11.557861,73020.0
2019-12-31,1.017429e+08,2.762114e+07,1.190067e+07,14558.000,4495524.864,5978556.928,0,4074127.104,512532.000,645171.968,...,5.481146e+07,0,2.087427e+07,0.000,-7.227446e+07,0.000,0.000,0,18.102791,115964.0
2019-09-30,1.024753e+08,2.900403e+07,1.501691e+07,14304.000,4236257.024,5978811.904,0,2545863.936,293060.000,918814.976,...,5.477930e+07,0,1.534137e+07,8665335.808,-7.090100e+07,0.000,0.000,0,18.187490,104745.0
2019-06-30,9.873898e+07,2.735604e+07,1.423340e+07,14038.000,4103118.080,6197564.928,0,1542556.032,507216.000,758140.992,...,5.481167e+07,0,1.534137e+07,5972242.944,-7.230108e+07,0.000,0.000,0,16.902554,100967.0
2019-03-31,9.690554e+07,2.619414e+07,1.282252e+07,13772.000,3980359.936,6032743.936,0,1980342.016,591273.984,773121.024,...,5.474039e+07,0,1.534137e+07,3023304.960,-7.179538e+07,0.000,0.000,0,15.901062,95415.0
2018-12-31,9.412614e+07,2.532960e+07,1.146350e+07,13391.000,4879256.064,5401793.024,0,2148713.984,741222.016,681731.008,...,5.478119e+07,0,1.543409e+07,0.000,-7.158487e+07,0.000,0.000,0,14.531096,87887.0


In [ ]:
# Para cada empresa nos fundamentos
for empresa in fundamentos:
    fundamento = fundamentos[empresa]
    fundamento = fundamento.sort_index()

    # Para cada coluna do DataFrame da empresa
    for coluna in fundamento:

        # Se for 'Adj Close' ou 'IBOV', calcula retorno percentual e decisão
        if 'Adj Close' in coluna or 'IBOV' in coluna:
            pass
        else:
            # Para demais indicadores, transforma em variação percentual ou sinaliza mudanças importantes
            condicoes = [
                (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
            ]
            valores = [
                -1,  # virou negativo
                1,   # virou positivo
                (abs(fundamento[coluna].shift(1)) - abs(fundamento[coluna])) / abs(fundamento[coluna].shift(1)),
                1,   # saiu de zero para positivo
                -1,  # saiu de zero para negativo
                1    # saiu de negativo para zero
            ]

            # Calcula variação percentual padrão se nenhuma condição acima for atendida
            fundamento[coluna] = np.select(
                condicoes,
                valores,
                default=fundamento[coluna] / fundamento[coluna].shift(1) - 1
            )

    # Calcula retorno percentual do próximo dia em relação ao atual
    fundamento['Adj Close'] = fundamento['Adj Close'].shift(-1) / fundamento['Adj Close'] - 1
    fundamento['IBOV'] = fundamento['IBOV'].shift(-1) / fundamento['IBOV'] - 1

    # Calcula o resultado relativo ao Ibovespa
    fundamento['Resultado'] = fundamento['Adj Close'] - fundamento['IBOV']

    # Define decisão:
    # 2: outperform (melhor que o IBOV)
    # 1: desempenho inferior até -2%
    # 0: desempenho abaixo de -2%
    condicoes = [
        (fundamento['Resultado'] > 0),
        (fundamento['Resultado'] < 0) & (fundamento['Resultado'] >= -0.02),
        (fundamento['Resultado'] < -0.02),
    ]
    valores = [2, 1, 0]
    fundamento['Decisao'] = np.select(condicoes, valores)

    # Atualiza o DataFrame no dicionário
    fundamentos[empresa] = fundamento

# Exibe resultado final da ABEV3 como exemplo
display(fundamentos['ABEV3'])

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber_1,Estoques_1,Ativos Biológicos_1,Tributos a Recuperar,Despesas Antecipadas_1,Outros Ativos Circulantes,...,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital,Adj Close,IBOV,Resultado,Decisao
ABEV3,,,,,,,,,,,,,,,,,,,,,
2012-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.013860,-0.075469,0.061609,2
2013-03-31,-0.004402,0.082509,0.540941,NaN,-0.855514,NaN,NaN,-0.878257,NaN,NaN,...,-1.000000,1.000000,NaN,NaN,-0.022593,NaN,0.011719,-0.157847,0.169566,2
2013-06-30,42.823044,154.470551,59.403415,1.000000,895.341086,1.000000,NaN,3040.654255,1.000000,1.000000,...,1.000000,36.638618,1.000000,1.000000,-1.000000,NaN,0.029640,0.102851,-0.073211,0
2013-09-30,0.014976,0.034944,0.078755,0.259921,-0.012680,-0.051246,NaN,0.150925,-0.053987,0.184512,...,0.000000,0.690105,-1.000000,-0.016194,-0.035818,NaN,0.022673,-0.015878,0.038551,2
2013-12-31,0.151882,0.640439,1.334113,-0.528801,0.315030,0.080784,NaN,0.642232,0.326698,0.198435,...,4.610997,-1.000000,0.002089,-1.000000,1.000000,NaN,-0.012411,-0.021201,0.008790,2
2014-03-31,-0.078297,-0.201158,-0.353510,0.421231,-0.139316,0.095442,NaN,0.016833,-0.133515,0.226973,...,-0.683412,1.000000,-0.018404,NaN,NaN,NaN,-0.066860,0.054607,-0.121467,0
2014-06-30,-0.004866,-0.035410,-0.140116,-0.073713,0.137713,0.018993,NaN,0.063286,-0.189384,0.071060,...,0.000000,1.519628,-0.003955,NaN,NaN,NaN,0.027849,0.017830,0.010019,2
2014-09-30,0.033925,-0.020712,-0.083799,0.386513,-0.074255,0.012343,NaN,0.129495,-0.024556,0.237706,...,0.000000,0.084282,0.014455,NaN,NaN,NaN,0.030113,-0.075929,0.106043,2
2014-12-31,0.107749,0.341943,0.691348,0.353403,-0.075340,0.079871,NaN,-0.072745,0.448937,2.030331,...,1.633530,-1.000000,0.009100,NaN,NaN,NaN,0.219934,0.022857,0.197077,2


In [ ]:
# Lista de colunas da ABEV3 para verificar valores ausentes
colunas = list(fundamentos["ABEV3"].columns)

# Inicializa dicionário com zero para contar valores ausentes por coluna
valores_vazios = dict.fromkeys(colunas, 0)

# Contador do total de linhas processadas
total_linhas = 0

# Para cada empresa nos fundamentos
for empresa in fundamentos:
    tabela = fundamentos[empresa]
    total_linhas += tabela.shape[0]  # Soma total de linhas

    # Para cada coluna, conta valores ausentes (NaN) e acumula
    for coluna in colunas:
        qtde_vazios = pd.isnull(tabela[coluna]).sum()
        valores_vazios[coluna] += qtde_vazios

# Exibe a quantidade total de valores ausentes por coluna
print(valores_vazios)

# Exibe o total de linhas processadas
print(total_linhas)

{'Ativo Total': np.int64(61), 'Ativo Circulante': np.int64(61), 'Caixa e Equivalentes de Caixa': np.int64(61), 'Aplicações Financeiras': np.int64(623), 'Contas a Receber_1': np.int64(65), 'Estoques_1': np.int64(483), 'Ativos Biológicos_1': np.int64(1874), 'Tributos a Recuperar': np.int64(266), 'Despesas Antecipadas_1': np.int64(1059), 'Outros Ativos Circulantes': np.int64(281), 'Ativo Realizável a Longo Prazo': np.int64(61), 'Aplicações Financeiras Avaliadas a Valor Justo': np.int64(1581), 'Aplicações Financeiras Avaliadas ao Custo Amortizado': np.int64(1661), 'Contas a Receber': np.int64(937), 'Estoques': np.int64(1434), 'Ativos Biológicos': np.int64(1823), 'Tributos Diferidos_1': np.int64(822), 'Despesas Antecipadas': np.int64(1130), 'Créditos com Partes Relacionadas': np.int64(1271), 'Outros Ativos Não Circulantes': np.int64(426), 'Investimentos': np.int64(345), 'Imobilizado': np.int64(62), 'Intangível': np.int64(86), 'Diferido': np.int64(2004), 'Passivo Total': np.int64(61), 'Passi

In [ ]:
# Inicializa lista para armazenar colunas que serão removidas
remover_colunas = []

# Identifica colunas com mais de 1/3 de valores ausentes
for coluna in valores_vazios:
    if valores_vazios[coluna] > (total_linhas / 3):
        remover_colunas.append(coluna)

# Remove colunas identificadas e preenche demais valores ausentes com 0
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)  # Remove colunas
    fundamentos[empresa] = fundamentos[empresa].fillna(0)  # Preenche NaN restantes com 0

In [ ]:
# Remove colunas específicas relacionadas ao retorno e resultado das análises
for empresa in fundamentos:
    fundamentos[empresa] = fundamentos[empresa].drop(['Adj Close', 'IBOV', "Resultado"], axis=1)

# Exibe a nova dimensão (linhas, colunas) do DataFrame da empresa 'ABEV3'
print(fundamentos['ABEV3'].shape)

(33, 32)
